<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [9]:
import numpy as np
import pandas as pd
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ============================================================================
# FUNCTION 1: Memory-Efficient Clustering Algorithm
# ============================================================================

def identify_commission_rate_groups_memory_efficient(df,
                                                      transaction_col='transaction_amount',
                                                      commission_col='commission_fee',
                                                      min_rate_diff=0.001,
                                                      min_cluster_size=5):
    """
    Memory-efficient clustering using sorting instead of DBSCAN.

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing transaction data
    transaction_col : str
        Column name for transaction amounts
    commission_col : str
        Column name for commission fees
    min_rate_diff : float
        Minimum expected difference between commission rates (e.g., 0.002 = 0.2%)
    min_cluster_size : int
        Minimum number of transactions to form a valid cluster

    Returns:
    --------
    pandas.DataFrame
        Original DataFrame with added columns:
        - 'commission_rate': calculated ratio
        - 'cluster_id': cluster assignment (-1 for outliers)
        - 'cluster_commission_rate': median rate for the cluster
    """
    print(f"Processing {len(df):,} transactions...")

    # Step 1: Calculate rates
    start = time.time()
    rates = (df[commission_col] / df[transaction_col]).values

    # Filter valid rates
    valid_mask = (df[transaction_col] > 0) & np.isfinite(rates)
    valid_indices = np.where(valid_mask)[0]
    valid_rates = rates[valid_mask]

    step1_time = time.time() - start
    print(f"  [1] Calculate & filter rates: {step1_time:.2f}s ({len(valid_rates):,} valid)")

    # Step 2: Sort rates with indices
    start = time.time()
    sorted_idx = np.argsort(valid_rates)
    sorted_rates = valid_rates[sorted_idx]
    step2_time = time.time() - start
    print(f"  [2] Sort rates: {step2_time:.2f}s")

    # Step 3: Find clusters by scanning sorted rates
    start = time.time()
    labels = np.full(len(valid_rates), -1, dtype=np.int32)
    eps = min_rate_diff / 2
    cluster_id = 0

    i = 0
    while i < len(sorted_rates):
        # Find all points within eps of current point
        j = i
        while j < len(sorted_rates) and sorted_rates[j] - sorted_rates[i] <= eps:
            j += 1

        cluster_size = j - i

        # If enough points, form a cluster
        if cluster_size >= min_cluster_size:
            labels[sorted_idx[i:j]] = cluster_id
            cluster_id += 1
            i = j
        else:
            # Skip this point (remains outlier)
            i += 1

    step3_time = time.time() - start
    print(f"  [3] Clustering scan: {step3_time:.2f}s (found {cluster_id} clusters)")

    # Step 4: Map back to original dataframe
    start = time.time()
    all_labels = np.full(len(df), -1, dtype=np.int32)
    all_labels[valid_indices] = labels

    df['cluster_id'] = all_labels
    df['commission_rate'] = rates

    # Calculate cluster rates
    cluster_rates = {}
    for cid in range(cluster_id):
        mask = labels == cid
        cluster_rates[cid] = np.median(valid_rates[mask])

    df['cluster_commission_rate'] = df['cluster_id'].map(cluster_rates)

    step4_time = time.time() - start
    print(f"  [4] Map back to dataframe: {step4_time:.2f}s")

    total_time = step1_time + step2_time + step3_time + step4_time
    print(f"\n  ✓ Total clustering time: {total_time:.2f}s")

    return df


# ============================================================================
# FUNCTION 2: Summary Generation with Validation
# ============================================================================

def summarize_commission_groups(df_with_clusters, valid_rates=None, tolerance=0.0002):
    """
    Generate a summary of identified commission rate groups.

    Parameters:
    -----------
    df_with_clusters : pandas.DataFrame
        DataFrame returned by clustering function
    valid_rates : list of float, optional
        Known valid commission rates (e.g., [0.035, 0.038, 0.048, 0.050])
    tolerance : float
        How close a cluster rate must be to a valid rate to match (default: 0.1%)

    Returns:
    --------
    pandas.DataFrame
        Summary statistics for each cluster
    """
    summary = []

    for cluster_id in sorted(df_with_clusters['cluster_id'].unique()):
        cluster_data = df_with_clusters[df_with_clusters['cluster_id'] == cluster_id]

        if cluster_id == -1:
            label = 'Outliers'
            cluster_rate = np.nan
        else:
            label = f'Group {cluster_id}'
            cluster_rate = cluster_data['cluster_commission_rate'].iloc[0]

        # Check if this matches a valid rate
        is_valid = 'N/A'
        if valid_rates is not None and cluster_id != -1:
            for valid_rate in valid_rates:
                if abs(cluster_rate - valid_rate) <= tolerance:
                    is_valid = '✓ Valid'
                    break
            else:
                is_valid = '✗ Invalid'

        summary.append({
            'Group': label,
            'Cluster_ID': cluster_id,
            'Count': len(cluster_data),
            'Percentage': len(cluster_data) / len(df_with_clusters) * 100,
            'Commission_Rate_%': cluster_rate * 100 if not np.isnan(cluster_rate) else np.nan,
            'Rate_Std_Dev': cluster_data['commission_rate'].std(),
            'Min_Rate_%': cluster_data['commission_rate'].min() * 100,
            'Max_Rate_%': cluster_data['commission_rate'].max() * 100,
            'Valid_Rate': is_valid
        })

    return pd.DataFrame(summary)


# ============================================================================
# FUNCTION 3: Visualization
# ============================================================================

def visualize_commission_groups_lite(df_with_clusters, max_scatter_points=10000):
    """
    Memory-efficient visualization with sampling.

    Parameters:
    -----------
    df_with_clusters : pandas.DataFrame
        DataFrame returned by clustering function
    max_scatter_points : int
        Maximum number of points to plot in scatter plot (default: 10000)
    """
    print(f"\nGenerating visualizations...")
    start = time.time()

    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Distribution of Commission Rates by Group',
                       'Transaction Amount vs Commission Fee (sampled)'),
        horizontal_spacing=0.12
    )

    cluster_ids = sorted(df_with_clusters['cluster_id'].unique())

    # Plot 1: Histogram (uses all data efficiently via binning)
    all_rates = df_with_clusters['commission_rate'] * 100
    rate_min, rate_max = all_rates.min(), all_rates.max()
    bin_size = (rate_max - rate_min) / 30

    for cluster_id in cluster_ids:
        cluster_data = df_with_clusters[df_with_clusters['cluster_id'] == cluster_id]

        if cluster_id == -1:
            name = 'Outliers'
            opacity = 0.5
        else:
            rate = cluster_data['cluster_commission_rate'].iloc[0] * 100
            name = f'Group {cluster_id} ({rate:.2f}%)'
            opacity = 0.7

        fig.add_trace(
            go.Histogram(
                x=cluster_data['commission_rate'] * 100,
                name=name,
                opacity=opacity,
                xbins=dict(start=rate_min, end=rate_max, size=bin_size),
                legendgroup=f'g{cluster_id}',
            ),
            row=1, col=1
        )

    # Plot 2: Scatter (SAMPLED for memory efficiency)
    if len(df_with_clusters) > max_scatter_points:
        print(f"  Sampling {max_scatter_points:,} points for scatter plot (from {len(df_with_clusters):,} total)")
        sample_df = df_with_clusters.sample(n=max_scatter_points, random_state=42)
    else:
        sample_df = df_with_clusters

    for cluster_id in cluster_ids:
        cluster_data = sample_df[sample_df['cluster_id'] == cluster_id]

        if len(cluster_data) == 0:
            continue

        name = 'Outliers' if cluster_id == -1 else f'Group {cluster_id}'
        symbol = 'x' if cluster_id == -1 else 'circle'
        opacity = 0.5 if cluster_id == -1 else 0.6

        fig.add_trace(
            go.Scatter(
                x=cluster_data['transaction_amount'],
                y=cluster_data['commission_fee'],
                mode='markers',
                name=name,
                opacity=opacity,
                marker=dict(size=4, symbol=symbol),
                legendgroup=f'g{cluster_id}',
                showlegend=False
            ),
            row=1, col=2
        )

    fig.update_xaxes(title_text="Commission Rate (%)", row=1, col=1)
    fig.update_yaxes(title_text="Number of Transactions", row=1, col=1)
    fig.update_xaxes(title_text="Transaction Amount", row=1, col=2)
    fig.update_yaxes(title_text="Commission Fee", row=1, col=2)

    fig.update_layout(
        height=600,
        width=1400,
        barmode='overlay',
        template='plotly_white'
    )

    viz_time = time.time() - start
    print(f"  ✓ Visualization time: {viz_time:.2f}s")

    fig.show()


# ============================================================================
# EXAMPLE USAGE: Test with 100K transactions
# ============================================================================

if __name__ == "__main__":
    print("="*70)
    print("COMMISSION RATE CLUSTERING - COMPLETE EXAMPLE")
    print("="*70)

    # ========================================================================
    # STEP 1: Define your valid commission rates
    # ========================================================================

    valid_rates = [0.035, 0.038, 0.048, 0.050]  # 3.5%, 3.8%, 4.8%, 5.0%

    # Calculate minimum gap between valid rates
    gaps = [valid_rates[i+1] - valid_rates[i] for i in range(len(valid_rates) - 1)]
    min_rate_diff = min(gaps)

    print(f"\nValid commission rates: {[f'{r*100}%' for r in valid_rates]}")
    print(f"Minimum gap between rates: {min_rate_diff*100:.2f}%")
    print(f"Will use eps = {min_rate_diff/2*100:.2f}%\n")

    # ========================================================================
    # STEP 2: Generate test data (replace with your actual data)
    # ========================================================================

    print("[STEP 1] Generating test data...")
    data_gen_start = time.time()

    np.random.seed(42)
    n = 100_000

    amounts = np.random.uniform(100, 10000, n)
    fees = np.zeros(n)

    # Group 1: 3.5% (25k transactions)
    fees[:25_000] = amounts[:25_000] * 0.035 + np.random.normal(0, 1, 25_000)

    # Group 2: 3.8% (25k transactions)
    fees[25_000:50_000] = amounts[25_000:50_000] * 0.038 + np.random.normal(0, 1, 25_000)

    # Group 3: 4.8% (20k transactions)
    fees[50_000:70_000] = amounts[50_000:70_000] * 0.048 + np.random.normal(0, 1, 20_000)

    # Group 4: 5.0% (20k transactions)
    fees[70_000:90_000] = amounts[70_000:90_000] * 0.050 + np.random.normal(0, 1, 20_000)

    # Outliers (10k transactions with random rates)
    fees[90_000:] = amounts[90_000:] * np.random.uniform(0.02, 0.07, 10_000)

    # Shuffle data
    idx = np.random.permutation(n)
    df = pd.DataFrame({
        'transaction_amount': amounts[idx],
        'commission_fee': fees[idx]
    })

    data_gen_time = time.time() - data_gen_start
    print(f"✓ Data generation complete: {data_gen_time:.2f}s")
    print(f"  Dataset size: {len(df):,} transactions\n")

    # ========================================================================
    # STEP 3: Run clustering
    # ========================================================================

    print("[STEP 2] Running clustering algorithm...")
    print("-" * 70)
    clustering_start = time.time()

    df_clustered = identify_commission_rate_groups_memory_efficient(
        df,
        transaction_col='transaction_amount',
        commission_col='commission_fee',
        min_rate_diff=min_rate_diff,
        min_cluster_size=10
    )

    clustering_total_time = time.time() - clustering_start
    print("-" * 70)
    print(f"✓ Clustering complete!\n")

    # ========================================================================
    # STEP 4: Generate summary with validation
    # ========================================================================

    print("[STEP 3] Generating summary...")
    summary_start = time.time()

    summary = summarize_commission_groups(
        df_clustered,
        valid_rates=valid_rates,
        tolerance=0.0001  # 0.01% tolerance
    )

    summary_time = time.time() - summary_start

    print("\n" + "="*70)
    print("COMMISSION RATE GROUPS SUMMARY")
    print("="*70)
    print(summary.to_string(index=False, float_format=lambda x: f'{x:.2f}'))

    # Additional statistics
    n_valid_groups = summary[summary['Valid_Rate'] == '✓ Valid'].shape[0]
    n_invalid_groups = summary[summary['Valid_Rate'] == '✗ Invalid'].shape[0]
    n_outliers = (df_clustered['cluster_id'] == -1).sum()

    print(f"\n" + "="*70)
    print("VALIDATION SUMMARY")
    print("="*70)
    print(f"Valid groups found:   {n_valid_groups}")
    print(f"Invalid groups found: {n_invalid_groups}")
    print(f"Total outliers:       {n_outliers:,} ({n_outliers/len(df)*100:.1f}%)")

    # ========================================================================
    # STEP 5: Visualize
    # ========================================================================

    print(f"\n[STEP 4] Creating visualizations...")
    viz_start = time.time()
    visualize_commission_groups_lite(df_clustered, max_scatter_points=10000)
    viz_total_time = time.time() - viz_start

    # ========================================================================
    # Final timing summary
    # ========================================================================

    print("\n" + "="*70)
    print("TIMING SUMMARY")
    print("="*70)
    print(f"Data generation:     {data_gen_time:6.2f}s")
    print(f"Clustering:          {clustering_total_time:6.2f}s  ← MAIN ALGORITHM")
    print(f"Summary generation:  {summary_time:6.2f}s")
    print(f"Visualization:       {viz_total_time:6.2f}s")
    print("-" * 70)
    total_time = data_gen_time + clustering_total_time + summary_time + viz_total_time
    print(f"TOTAL:               {total_time:6.2f}s")
    print("="*70)

    print(f"\nPerformance metrics:")
    print(f"  Throughput: {n/clustering_total_time:,.0f} transactions/second")
    print(f"  Clustering % of total: {clustering_total_time/total_time*100:.1f}%")

COMMISSION RATE CLUSTERING - COMPLETE EXAMPLE

Valid commission rates: ['3.5000000000000004%', '3.8%', '4.8%', '5.0%']
Minimum gap between rates: 0.20%
Will use eps = 0.10%

[STEP 1] Generating test data...
✓ Data generation complete: 0.01s
  Dataset size: 100,000 transactions

[STEP 2] Running clustering algorithm...
----------------------------------------------------------------------
Processing 100,000 transactions...
  [1] Calculate & filter rates: 0.00s (100,000 valid)
  [2] Sort rates: 0.00s
  [3] Clustering scan: 0.06s (found 51 clusters)
  [4] Map back to dataframe: 0.01s

  ✓ Total clustering time: 0.07s
----------------------------------------------------------------------
✓ Clustering complete!

[STEP 3] Generating summary...

COMMISSION RATE GROUPS SUMMARY
   Group  Cluster_ID  Count  Percentage  Commission_Rate_%  Rate_Std_Dev  Min_Rate_%  Max_Rate_% Valid_Rate
Outliers          -1      1        0.00                NaN           NaN        1.39        1.39        N/A
 Gro


TIMING SUMMARY
Data generation:       0.01s
Clustering:            0.07s  ← MAIN ALGORITHM
Summary generation:    0.03s
Visualization:         0.19s
----------------------------------------------------------------------
TOTAL:                 0.31s

Performance metrics:
  Throughput: 1,369,305 transactions/second
  Clustering % of total: 23.9%


EXPERIMENTS

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

def identify_commission_rate_groups(df, transaction_col='transaction_amount',
                                     commission_col='commission_fee',
                                     min_rate_diff=0.001,  # 0.1% minimum difference
                                     min_cluster_size=5):
    """
    Identify groups of transactions by commission rates using DBSCAN.

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing transaction data
    transaction_col : str
        Column name for transaction amounts
    commission_col : str
        Column name for commission fees
    min_rate_diff : float
        Minimum expected difference between commission rates (default: 0.001 = 0.1%)
    min_cluster_size : int
        Minimum number of transactions to form a valid cluster

    Returns:
    --------
    pandas.DataFrame
        Original DataFrame with added columns:
        - 'commission_rate': calculated ratio
        - 'cluster_id': cluster assignment (-1 for outliers)
        - 'cluster_commission_rate': median rate for the cluster
    """

    # Create a copy to avoid modifying original
    result_df = df.copy()

    # Calculate commission rates (ratios)
    result_df['commission_rate'] = (
        result_df[commission_col] / result_df[transaction_col]
    )

    # Handle edge cases
    # Remove transactions with zero or negative amounts
    valid_mask = result_df[transaction_col] > 0
    result_df = result_df[valid_mask].copy()

    # Prepare data for clustering (reshape to 2D array)
    X = result_df['commission_rate'].values.reshape(-1, 1)

    # Set eps based on minimum rate difference
    # Use half of min_rate_diff to ensure we don't merge distinct rates
    eps = min_rate_diff / 2

    # Apply DBSCAN
    # eps: maximum distance for points to be in same neighborhood
    # min_samples: minimum points to form a dense region (cluster)
    dbscan = DBSCAN(eps=eps, min_samples=min_cluster_size)
    result_df['cluster_id'] = dbscan.fit_predict(X)

    # Calculate median commission rate for each cluster
    cluster_rates = {}
    for cluster_id in result_df['cluster_id'].unique():
        if cluster_id == -1:
            continue  # Skip outliers

        cluster_mask = result_df['cluster_id'] == cluster_id
        cluster_rates[cluster_id] = result_df.loc[cluster_mask, 'commission_rate'].median()

    # Assign cluster rates
    result_df['cluster_commission_rate'] = result_df['cluster_id'].map(cluster_rates)

    # For outliers, set to NaN
    result_df.loc[result_df['cluster_id'] == -1, 'cluster_commission_rate'] = np.nan

    return result_df


def summarize_commission_groups(df_with_clusters):
    """
    Generate a summary of identified commission rate groups.

    Parameters:
    -----------
    df_with_clusters : pandas.DataFrame
        DataFrame returned by identify_commission_rate_groups

    Returns:
    --------
    pandas.DataFrame
        Summary statistics for each cluster
    """
    summary = []

    for cluster_id in sorted(df_with_clusters['cluster_id'].unique()):
        cluster_data = df_with_clusters[df_with_clusters['cluster_id'] == cluster_id]

        if cluster_id == -1:
            label = 'Outliers'
        else:
            label = f'Group {cluster_id}'

        summary.append({
            'Group': label,
            'Cluster_ID': cluster_id,
            'Count': len(cluster_data),
            'Commission_Rate_%': cluster_data['cluster_commission_rate'].iloc[0] * 100
                                 if cluster_id != -1 else np.nan,
            'Rate_Std_Dev': cluster_data['commission_rate'].std(),
            'Min_Rate_%': cluster_data['commission_rate'].min() * 100,
            'Max_Rate_%': cluster_data['commission_rate'].max() * 100,
        })

    return pd.DataFrame(summary)


def visualize_commission_groups(df_with_clusters, figsize=(14, 6)):
    """
    Visualize the identified commission rate groups.

    Parameters:
    -----------
    df_with_clusters : pandas.DataFrame
        DataFrame returned by identify_commission_rate_groups
    figsize : tuple
        Figure size (width, height)
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)

    # Plot 1: Histogram of commission rates colored by cluster
    for cluster_id in sorted(df_with_clusters['cluster_id'].unique()):
        cluster_data = df_with_clusters[df_with_clusters['cluster_id'] == cluster_id]

        if cluster_id == -1:
            label = 'Outliers'
            alpha = 0.5
        else:
            label = f'Group {cluster_id} ({cluster_data["cluster_commission_rate"].iloc[0]*100:.2f}%)'
            alpha = 0.7

        ax1.hist(cluster_data['commission_rate'] * 100,
                bins=50, alpha=alpha, label=label)

    ax1.set_xlabel('Commission Rate (%)')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Distribution of Commission Rates by Group')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Plot 2: Scatter plot of transaction amount vs commission fee
    for cluster_id in sorted(df_with_clusters['cluster_id'].unique()):
        cluster_data = df_with_clusters[df_with_clusters['cluster_id'] == cluster_id]

        if cluster_id == -1:
            label = 'Outliers'
            marker = 'x'
            alpha = 0.5
        else:
            label = f'Group {cluster_id}'
            marker = 'o'
            alpha = 0.6

        ax2.scatter(cluster_data['transaction_amount'],
                   cluster_data['commission_fee'],
                   alpha=alpha, label=label, marker=marker, s=20)

    ax2.set_xlabel('Transaction Amount')
    ax2.set_ylabel('Commission Fee')
    ax2.set_title('Transaction Amount vs Commission Fee')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

In [ ]:
# Generate sample data with multiple commission rates and outliers
np.random.seed(42)

n_transactions = 1000

# Create transactions with different commission rates
# Group 1: 3.5% commission (350 transactions)
group1 = pd.DataFrame({
    'transaction_amount': np.random.uniform(100, 10000, 350),
})
group1['commission_fee'] = group1['transaction_amount'] * 0.035 + np.random.normal(0, 0.5, 350)

# Group 2: 3.8% commission (300 transactions)
group2 = pd.DataFrame({
    'transaction_amount': np.random.uniform(100, 10000, 300),
})
group2['commission_fee'] = group2['transaction_amount'] * 0.038 + np.random.normal(0, 0.5, 300)

# Group 3: 4.1% commission (250 transactions)
group3 = pd.DataFrame({
    'transaction_amount': np.random.uniform(100, 10000, 250),
})
group3['commission_fee'] = group3['transaction_amount'] * 0.041 + np.random.normal(0, 0.5, 250)

# Add outliers (100 transactions with random rates)
outliers = pd.DataFrame({
    'transaction_amount': np.random.uniform(100, 10000, 100),
})
outliers['commission_fee'] = outliers['transaction_amount'] * np.random.uniform(0.02, 0.06, 100)

# Combine all data
df = pd.concat([group1, group2, group3, outliers], ignore_index=True)

# Shuffle the data
df = df.sample(frac=1).reset_index(drop=True)

print("Original data shape:", df.shape)
print("\nFirst few rows:")
print(df.head())

# Identify commission rate groups
df_clustered = identify_commission_rate_groups(
    df,
    min_rate_diff=0.001,  # 0.1% minimum difference
    min_cluster_size=10
)

# Generate summary
summary = summarize_commission_groups(df_clustered)
print("\n" + "="*70)
print("COMMISSION RATE GROUPS SUMMARY")
print("="*70)
print(summary.to_string(index=False))

# Count outliers
n_outliers = (df_clustered['cluster_id'] == -1).sum()
print(f"\nTotal outliers detected: {n_outliers} ({n_outliers/len(df)*100:.1f}%)")

# Visualize results
visualize_commission_groups(df_clustered)

# Show some example outliers
if n_outliers > 0:
    print("\nExample outliers:")
    outlier_examples = df_clustered[df_clustered['cluster_id'] == -1].head()
    print(outlier_examples[['transaction_amount', 'commission_fee', 'commission_rate']])